# Segmentation and Tracking

This notebook demonstrates a standard segmentation and tracking workflow.
It loads a vector of example images,
segments them,
then runs a tracking algorithm to match floes in the images.

In this example, we use two images.
However, the same syntax can be used for a larger number of images. 


Set up the environment.

In [ ]:
using Pkg
Pkg.add("IceFloeTracker")
Pkg.add("Images")

Load packages and functions required. 

In [ ]:
using IceFloeTracker: LopezAcosta2019Tiling, Watkins2025GitHub, FloeTracker, ChainedFilterFunction, MinimumWeightMatchingFunction
using Images: erode, segment_mean, labels_map, SegmentedImage, RGB, mosaicview
using Dates: DateTime

Load the images to be segmented. 
In this case we use data from the Watkins 2026 Ice Floe Validation dataset.

In [ ]:
data_loader = Watkins2025GitHub(; ref="25ba4d46814a5423b65ad675aaec05633d17a37e")
cases = data_loader(c -> c.case_number == 6)
mosaicview([c.modis_truecolor for c in cases], nrow=1)

The metadata for the cases are:

In [ ]:
cases.metadata

Run segmentation on the images.

In [ ]:
segmenter = LopezAcosta2019Tiling.Segment()
tc_images = [c.modis_truecolor for c in cases] # TODO: Update validation dataset so that `cases.modis_truecolor` does this
fc_images = [c.modis_falsecolor for c in cases] # TODO: Update validation dataset so that `cases.modis_falsecolor` does this
lm_images = [c.modis_landmask for c in cases] # TODO: Update validation dataset so that `cases.modis_landmask` does this
segmentation_results = segmenter.(tc_images, fc_images, lm_images)

Show the segmentation results:

In [ ]:
segmentation_results

Run the tracker on the segmentation results.
The tracker takes a vector of segmented images 
and requires the observation time of each for filtering candidate matches.

In [ ]:
tracker = FloeTracker(
    filter_function=FilterFunction(), 
    matching_function=MinimumWeightMatchingFunction()
)
tracking_results = tracker(segmentation_results, cases.metadata.pass_time)

The tracking results are returned as a dataframe of floes and their matches. 

In [ ]:
tracking_results